# Progetto Introduzione all'intelligenza artificiale

Il progetto consiste nella realizzazione di una applicazione di Intelligenza Artificiale completa degli
aspetti di gestione di: sensori per l’acquisizione dei dati dall’esterno relativi a stati e obiettivi,
ragionamento/ricerca della soluzione per i goal acquisiti, esecutori per la realizzazione delle azioni
che conducono alla soluzione.

## Progetto Uniform Coloring
Uniform Coloring è un dominio in cui si hanno a disposizione alcune celle da colorare, e vari colori a disposizione. 
Per semplicità immaginiamo una griglia rettangolare in cui è possibile spostare una testina colorante fra le celle attigue secondo le 4 direzioni cardinali (N,S,E,W), senza uscire dalla griglia. 

Tutte le celle hanno un colore di partenza (B=blu, Y=yellow, G=green) ad eccezione di quella in cui si trova la testina indicata con T. La testina può colorare la cella in cui si trova con uno qualsiasi dei colori disponibili a differenti costi (cost(B)=1, cost(Y)=2, cost(G)=3), mentre gli spostamenti hanno tutti costo uniforme pari a 1. 

>L’obiettivo è colorare tutte le celle dello stesso colore (non importa quale) e riportare la testina nella sua posizione di partenza.

La codifica di tutto il dominio (topologia della griglia, definizione delle azioni etc.) è parte dell’esercizio. Partendo dalla posizione iniziale della testina e combinando azioni di spostamento e colorazione, si chiede di trovare la sequenza di azioni dell’agente per raggiungere l’obiettivo.

La posizione iniziale della testina, la struttura della griglia e la colorazione iniziale delle celle sono passati al sistema tramite un’immagine.

### Inizializzaione delle librerie e moduli

In [ ]:
from emnist import list_datasets
import tensorflow as tf

list_datasets()

# print some images from the dataset to test
import matplotlib.pyplot as plt
from emnist import extract_training_samples
images, labels = extract_training_samples('letters')
for i in range(100):
    plt.subplot(10, 10, i+1)
    plt.axis('off')
    plt.imshow(images[i], cmap='gray')
plt.show()
